In [ ]:
#Importing the necessary libraries
import pandas as pd
import string
import numpy as np
import json
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
#import keras.utils as ku
from keras.utils import np_utils as ku 

import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
#Loading the dataset
#load all the datasets 
df1 = pd.read_csv('E:\ML\ML Excercise\Assignment\CAvideos.csv')
df2 = pd.read_csv('E:\ML\ML Excercise\Assignment\DEvideos.csv')
df3 = pd.read_csv('E:\ML\ML Excercise\Assignment\FRvideos.csv')
df4 = pd.read_csv('E:\ML\ML Excercise\Assignment\GBvideos.csv')
df5 = pd.read_csv('E:\ML\ML Excercise\Assignment\INvideos.csv')
df6 = pd.read_csv('E:\ML\ML Excercise\Assignment\JPvideos.csv')
df7 = pd.read_csv('E:\ML\ML Excercise\Assignment\KRvideos.csv')
df8 = pd.read_csv('E:\ML\ML Excercise\Assignment\MXvideos.csv')
df9 = pd.read_csv('E:\ML\ML Excercise\Assignment\RUvideos.csv')
df10 = pd.read_csv('E:\ML\ML Excercise\Assignment\RSvideos.csv')

#load the datasets containing the category names
data1 = json.load(open('E:\ML\ML Excercise\Assignment\CA_category_id.json'))
data2 = json.load(open('E:\ML\ML Excercise\Assignment\DE_category_id.json'))
data3 = json.load(open('E:\ML\ML Excercise\Assignment\FR_category_id.json'))
data4 = json.load(open('E:\ML\ML Excercise\Assignment\GB_category_id.json'))
data5 = json.load(open('E:\ML\ML Excercise\Assignment\IN_category_id.json'))
data6 = json.load(open('E:\ML\ML Excercise\Assignment\JP_category_id.json'))
data7 = json.load(open('E:\ML\ML Excercise\Assignment\KR_category_id.json'))
data8 = json.load(open('E:\ML\ML Excercise\Assignment\MX_category_id.json'))
data9 = json.load(open('E:\ML\ML Excercise\Assignment\RU_category_id.json'))
data10 = json.load(open('E:\ML\ML Excercise\Assignment\RS_category_id.json'))

In [ ]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

#create a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))
df4['category_title'] = df4['category_id'].map(category_extractor(data4))
df5['category_title'] = df5['category_id'].map(category_extractor(data5))
df6['category_title'] = df6['category_id'].map(category_extractor(data6))
df7['category_title'] = df7['category_id'].map(category_extractor(data7))
df8['category_title'] = df8['category_id'].map(category_extractor(data8))
df9['category_title'] = df9['category_id'].map(category_extractor(data9))
df10['category_title'] = df10['category_id'].map(category_extractor(data10))

#join the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')
#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()

#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

In [ ]:
#Generating sequences
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
 
  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
   token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
   n_gram_sequence = token_list[:i+1]
   input_sequences.append(n_gram_sequence)
 
  return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes = total_words)
    return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
 
  # Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))
 
  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))
 
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
 
  return model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=20, verbose=5)

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      #predicted = model.predict_classes(token_list, verbose=0)
      predict_x=model.predict(token_list) 
      predicted=np.argmax(predict_x,axis=1)
 
      output_word = ""
      for word,index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              break
  seed_text += " "+output_word
  return seed_text.title()

print(generate_text("HAPPY", 5, model, max_sequence_len))